In [ ]:
import itertools
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import sklearn.datasets as skd
from sklearn import preprocessing
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.model_selection import (
    train_test_split, StratifiedShuffleSplit, 
    ShuffleSplit, KFold, StratifiedKFold,
)

import kditransform

In [ ]:
def get_wine():
    columns = [
        'Class label',
        'Alcohol', 'Malic acid', 'Ash', 'Ash alcalinity',
        'Magnesium', 'Total phenols', 'Flavanoids',
        'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 
        'Hue', 'OD280/OD315 of diluted wines', 'Proline'
    ]
    df = pd.io.parsers.read_csv('wine_data.csv', header=None)
    X_wine = df.values[:,1:]
    y_wine = df.values[:,0]
    return X_wine, y_wine

def get_iris():
    df = sns.load_dataset('iris')
    X_iris = df.values[:, 0:4]
    y_iris = df.values[:, 4]
    return X_iris, y_iris

def get_penguins():
    df = sns.load_dataset('penguins')
    df = df.dropna()
    columns = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
    X_penguins = df[columns].values
    y_penguins = df[['species']].values.flatten()
    return X_penguins, y_penguins

def get_hawks():
    columns = ['Wing', 'Weight', 'Culmen', 'Hallux', 'Tail']
    # Downloaded from https://vincentarelbundock.github.io/Rdatasets/csv/Stat2Data/Hawks.csv
    df = pd.io.parsers.read_csv('Hawks.csv')
    df = df[columns+['Species']]
    df = df.dropna()
    X_hawks = df[columns].values
    y_hawks = df[['Species']].values.flatten()
    return X_hawks, y_hawks

In [ ]:
n_random = 100
n_alphas = 20
n_splits = 30
outer_test_size = 0.3
inner_test_size = 0.3

datasets = [
    ("wine", get_wine()),
    ("iris", get_iris()),
    ("penguins", get_penguins()),
    ("hawks", get_hawks()),
]
methods = [
    (preprocessing.MinMaxScaler, 'min-max'),
    (preprocessing.QuantileTransformer, 'quantile'),
    (kditransform.KDITransformer, 'KD-integral'),
]
test_dict = {}
alpha_dict = {}
for dname, (X_all, y_all) in datasets:
    test_dict[dname] = {}
    alpha_dict[dname] = {}
    for mname in [mname for _, mname in methods]+['KD-integral (bwf=CV)']:
        test_dict[dname][mname] = {'pc': np.zeros(n_random)}
    alpha_dict[dname] = {'pc': np.zeros((n_random, n_alphas))}

for dname, (X_all, y_all) in datasets:
    print(dname)
    alphas = list(np.geomspace(0.1, 10, n_alphas))
    random_states = list(range(n_random))
    for rix, rstate in enumerate(random_states):
        X_train, X_test, y_train, y_test = train_test_split(
            X_all, y_all, test_size=outer_test_size, random_state=rstate)
          
        for preprocessor, pname in methods:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                prepper = preprocessor().fit(X_train)
                X_train_prepped = prepper.transform(X_train)
                X_test_prepped = prepper.transform(X_test)
            # PCA + GaussianNB
            pcaer = PCA(n_components=2).fit(X_train_prepped)
            X_train_prepped_pc = pcaer.transform(X_train_prepped)
            X_test_prepped_pc = pcaer.transform(X_test_prepped)
            gnb_pc = GaussianNB().fit(X_train_prepped_pc, y_train)
            acc_test_pc = metrics.accuracy_score(y_test, gnb_pc.predict(X_test_prepped_pc))
            test_dict[dname][pname]['pc'][rix] = acc_test_pc
    
        sss = ShuffleSplit(n_splits=n_splits, test_size=inner_test_size, random_state=rstate)
        acc_allsplits_pc = np.zeros((n_splits, len(alphas)))
        for sssi, (trtr_index, trdev_index) in enumerate(sss.split(X_train, y_train)):
            X_trtr = X_train[trtr_index, :]
            y_trtr = y_train[trtr_index]
            X_trdev = X_train[trdev_index, :]
            y_trdev = y_train[trdev_index]
            for aix, alpha in enumerate(alphas):
                prepper = kditransform.KDITransformer(alpha=alpha).fit(X_train)
                X_train_prepped = prepper.transform(X_train)
                X_trtr_prepped = prepper.transform(X_trtr)
                X_trdev_prepped = prepper.transform(X_trdev)
                pcaer = PCA(n_components=2).fit(X_train_prepped)
                X_trtr_prepped_pc = pcaer.transform(X_trtr_prepped)
                X_trdev_prepped_pc = pcaer.transform(X_trdev_prepped)
                gnb_pc = GaussianNB().fit(X_trtr_prepped_pc, y_trtr)
                acc_trdev_pc = metrics.accuracy_score(y_trdev, gnb_pc.predict(X_trdev_prepped_pc))
                acc_allsplits_pc[sssi, aix] = acc_trdev_pc
        acc_splits_pc = np.mean(acc_allsplits_pc, axis=0)
    
        for aix, alpha in enumerate(alphas):
            prepper = kditransform.KDITransformer(alpha=alpha).fit(X_train)
            X_train_prepped = prepper.transform(X_train)
            X_test_prepped = prepper.transform(X_test)
            pcaer = PCA(n_components=2).fit(X_train_prepped)
            X_train_prepped_pc = pcaer.transform(X_train_prepped)
            X_test_prepped_pc = pcaer.transform(X_test_prepped)
            gnb_pc = GaussianNB().fit(X_train_prepped_pc, y_train)
            acc_test_pc = metrics.accuracy_score(y_test, gnb_pc.predict(X_test_prepped_pc))
            alpha_dict[dname]['pc'][rix, aix] = acc_test_pc
            if aix == np.argmax(acc_splits_pc):
                test_dict[dname]['KD-integral (bwf=CV)']['pc'][rix] = acc_test_pc 

In [ ]:
for dname in list(test_dict.keys()):
    markers = itertools.cycle(('^', 's', '*', 'o', '.'))
    colors = itertools.cycle(('red', 'blue', 'purple', 'magenta', 'green')) 
    alph = np.array([np.min(alphas), np.max(alphas)]);
    fig = plt.figure(figsize=(4,3), dpi=150)
    for pname in list(test_dict[dname].keys()):
        tmean = test_dict[dname][pname]['pc'].mean()
        tstd = np.std(test_dict[dname][pname]['pc'], ddof=1) / np.sqrt(n_random)
        (_, caps, _) = plt.errorbar(
            alph, tmean*np.ones(2), yerr=tstd*np.ones(2), 
            marker=next(markers), c = next(colors), markersize=10, alpha=0.5, capsize=4,
            label=f'{pname} (bwf=1)' if pname=='KD-integral' else pname);
        for cap in caps:
            cap.set_markeredgewidth(2)
    amean = alpha_dict[dname]['pc'].mean(axis=0)
    astd = np.std(alpha_dict[dname]['pc'], axis=0, ddof=1) / np.sqrt(n_random)
    plt.errorbar(
        alphas, amean, yerr=astd,
        marker=next(markers), c=next(colors), markersize=2, alpha=0.5, capsize=2,
        label='KD-integral');
    plt.xlabel('KD-integral bandwidth factor');
    plt.ylabel('Accuracy');
    plt.xscale('log');
    plt.xlim(np.min(alphas), np.max(alphas));
    fig.savefig(f'Accuracy-vs-bwf-{dname}-pca-nolegend.pdf', bbox_inches='tight')
    plt.legend(loc='lower center');
    fig.savefig(f'Accuracy-vs-bwf-{dname}-pca.pdf', bbox_inches='tight')
    plt.title(f'{dname} PCA');

In [ ]:
"""
import pickle
with open("ansur_test_dict.pickle", "wb") as f:
    pickle.dump(test_dict, f)
with open("ansur_alpha_dict.pickle", "wb") as f:
    pickle.dump(alpha_dict, f)
""";